In [ ]:
'''
File structure reference, need to modify
arxiv_project/
  ├─ requirements.txt
  ├─ scripts/
  │   ├─ data_preprocessing.py #Clean, transform JSON -> CSV
  │   ├─ embedding_script.py #Generate embeddings via Transformers
  │   ├─ train_preparation.py #Convert embeddings to numeric CSV
  │   ├─ cluster_evaluation.py #Evaluate silhouette, Davies-Bouldin
  │   ├─ pipeline_definition.py #SageMaker Pipeline definition
  └─ buildspec.yml #For building up
'''

In [17]:
!pip install transformers torch scikit-learn boto3 sagemaker

In [14]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
from sagemaker.image_uris import retrieve
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.feature_store.feature_definition import FeatureDefinition, FeatureTypeEnum
import json
import pandas as pd
import os
from datetime import datetime

 - Because data is in a .json and needs to be converted to .csv for model

In [3]:
#REgion, role, session below
role = "arn:aws:iam::221082214706:role/MYLabRole" # Hardcode, maybe avoids errors
region = "us-east-1"

# Boto3 + SageMaker session 
boto_sess = boto3.Session(region_name=region)
session = sagemaker.Session(boto_session=boto_sess)
print(f"Using region: {region}")
print(f"USing role: {role}")

[01/31/25 01:02:19] INFO     Found credentials from IAM Role:                                   ]8;id=358840;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=391026;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

Using region: us-east-1
USing role: arn:aws:iam::221082214706:role/MYLabRole


In [5]:
# Setup S3 paths, chunk size, etc.
bucket_name = "arxiv-project-bucket"
raw_data_key = "arxiv-metadata-oai-snapshot.json"  # 4GB JSON file, need 4xlarge to parse
processed_prefix = "processed-data"
chunk_size = 200000 #  Reduce chunks of data flow in because, again, 1.7 million, 4GB dataset

Note: Bucket created already in previous steps, super easy, becareful

In [ ]:
Note: Data Ingestion, Script processing, etc.

In [6]:
raw_data_s3_uri = f"s3://{bucket_name}/{raw_data_key}"
processed_data_s3_uri = f"s3://{bucket_name}/{processed_prefix}/"

In [8]:
# Script Processor to run data_preprocessing.py file
processing_image_uri = retrieve(
    framework='sklearn',
    region=region,
    version='0.23-1'
)

script_processor = ScriptProcessor(
    image_uri=processing_image_uri,
    command=['python3'],
    role=role,
    instance_count=1,
    instance_type='ml.m5.4xlarge', # Need larger memory, maybe 8x size? uh oh to money
    volume_size_in_gb=200, # disk memory for temporary files pls work
    max_runtime_in_seconds=6*3600, # 6 hours runtimejust in case
    env={"AWS_DEFAULT_REGION": region}
)

[01/31/25 01:27:48] INFO     Defaulting to only available Python version: py3                     ]8;id=767210;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=361985;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#608\608]8;;\

                    INFO     Defaulting to only supported image scope: cpu.                       ]8;id=719427;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=375129;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#532\532]8;;\

                    INFO     Found credentials from IAM Role:                                   ]8;id=976781;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=759054;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [23]:
script_processor.run(
    code='data_preprocessing.py', # .py script with streaming/chunking logic
    inputs=[
        ProcessingInput(
            source=raw_data_s3_uri,
            destination="/opt/ml/processing/input"
        )
    ],
    outputs=[
        ProcessingOutput(
            source="/opt/ml/processing/output",
            destination=processed_data_s3_uri
        )
    ],
    arguments=[
        "--input-data", "/opt/ml/processing/input",
        "--output-data", "/opt/ml/processing/output",
        "--chunk-size", str(chunk_size)
    ]
)

[01/30/25 05:51:58] INFO     Creating processing-job with name                                      ]8;id=821993;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=555092;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1575\1575]8;;\
                             sagemaker-scikit-learn-2025-01-30-05-51-58-714                                        

..............Collecting transformers==4.26.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 100.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 103.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 133.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.1/670.1 kB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.6/761.6 kB 93.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 90.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.3 MB

In [24]:
# Inspect job hopefully actually done
job_desc = script_processor.jobs[-1].describe()
job_desc

{'ProcessingInputs': [{'InputName': 'input-1',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://arxiv-project-bucket/arxiv-metadata-oai-snapshot.json',
    'LocalPath': '/opt/ml/processing/input',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}},
  {'InputName': 'code',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-221082214706/sagemaker-scikit-learn-2025-01-30-05-51-58-714/input/code/data_preprocessing.py',
    'LocalPath': '/opt/ml/processing/input/code',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}}],
 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'output-1',
    'S3Output': {'S3Uri': 's3://arxiv-project-bucket/processed-data/',
     'LocalPath': '/opt/ml/processing/output',
     'S3UploadMode': 'EndOfJob'},
    'AppManaged': False}]},
 'ProcessingJobName': 'sa

In [26]:
!pip show sagemaker

Name: sagemaker
Version: 2.237.3
Summary: Open source library for training and deploying models on Amazon SageMaker.
Home-page: https://github.com/aws/sagemaker-python-sdk
Author: Amazon Web Services
Author-email: 
License: 
Location: /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages
Requires: attrs, boto3, cloudpickle, docker, fastapi, google-pasta, importlib-metadata, jsonschema, numpy, omegaconf, packaging, pandas, pathos, platformdirs, protobuf, psutil, pyyaml, requests, sagemaker-core, schema, smdebug-rulesconfig, tblib, tqdm, urllib3, uvicorn
Required-by: 


In [27]:
!pip install --upgrade sagemaker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.2 MB/s eta 0:00:00
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.237.3
    Uninstalling sagemaker-2.237.3:
      Successfully uninstalled sagemaker-2.237.3


In [39]:
!pip install --upgrade --force-reinstall sagemaker

  Using cached sagemaker-2.238.0-py3-none-any.whl.metadata (16 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached cloudpickle-3.1.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached importlib_metadata-6.11.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached pathos-0.3.3-py3-none-any.whl.metadata (11 kB)
  Using cached platformdirs-4.3.6-py3-none-any.whl.metadata (11 kB)
  Using cached protobuf-5.29.3-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
  Using cached psutil-6.1.1-cp36-abi3-manylinux_2_12_x86_64.manylinux2010_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (22 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata 

In [9]:
print("Actual runtime version:", sagemaker.__version__)

Actual runtime version: 2.237.3


In [21]:
# Feature Group name, record ID, event time, all the good stuff
# 2) Basic Feature Group parameters
feature_group_name = "arxiv-feature-group_v2"
record_identifier_name = "id"
event_time_name = "event_time"

# manual definition for features, cannot be dict because outdated sagemaker somehow
feature_definitions = [
    FeatureDefinition(feature_name="id", feature_type=FeatureTypeEnum.STRING),
    FeatureDefinition(feature_name="abstract", feature_type=FeatureTypeEnum.STRING),
    FeatureDefinition(feature_name="update_date",feature_type=FeatureTypeEnum.STRING),
    FeatureDefinition(feature_name="year", feature_type=FeatureTypeEnum.INTEGRAL),
    FeatureDefinition(feature_name="month", feature_type=FeatureTypeEnum.INTEGRAL),
    FeatureDefinition(feature_name="authors", feature_type=FeatureTypeEnum.STRING),
    FeatureDefinition(feature_name="event_time", feature_type=FeatureTypeEnum.STRING),
]

# FeatureGroup object
feature_group = FeatureGroup(
    name=feature_group_name,
    sagemaker_session=session
)
# Directly assign the feature definitions (instead of load_feature_definitions)
feature_group.feature_definitions = feature_definitions

s3_uri = f"s3://{bucket_name}/feature-store-offline/{feature_group_name}"


# 7) Now create the Feature Group
try:
    feature_group.create(
        s3_uri,
        record_identifier_name=record_identifier_name,
        event_time_feature_name=event_time_name,
        role_arn=role,
        enable_online_store=True,          
        # Store in the Online store for low-latency access
        # OfflineStoreConfig=offline_store_config comment out for now, somehow the version of Sagemaker 
        # Is outdated and different than the actual runtime nevironment somehow. 
        # May need to consider a forceful !pip install --upgrade --force-reinstall sagemaker
        # followed by importing and printing print("Version: ", sagemaker.__version__)
    )
    # Wait until the Feature Group is created
    print(f"Feature group '{feature_group_name}' created successfully!")
except Exception as e:
    print("Error creating Feature Group:", e)

Feature group 'arxiv-feature-group_v2' created successfully!


In [22]:
!aws sagemaker list-feature-groups --region us-east-1

{
    "FeatureGroupSummaries": [
        {
            "FeatureGroupName": "neighborhood-feature-group-26-03-30-30",
            "FeatureGroupArn": "arn:aws:sagemaker:us-east-1:221082214706:feature-group/neighborhood-feature-group-26-03-30-30",
            "CreationTime": 1737862233.01,
            "FeatureGroupStatus": "Created",
            "OfflineStoreStatus": {
                "Status": "Active"
            }
        },
        {
            "FeatureGroupName": "arxiv-feature-group_v2",
            "FeatureGroupArn": "arn:aws:sagemaker:us-east-1:221082214706:feature-group/arxiv-feature-group_v2",
            "CreationTime": 1738289451.937,
            "FeatureGroupStatus": "Creating"
        },
        {
            "FeatureGroupName": "arxiv-feature-group",
            "FeatureGroupArn": "arn:aws:sagemaker:us-east-1:221082214706:feature-group/arxiv-feature-group",
            "CreationTime": 1738288517.203,
            "FeatureGroupStatus": "Created"
        }
    ]
}
